# Deep Neural Network for MNIST Classification

In [1]:
# Import of required Python packages and datasets
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
# Datasets review (optionally)
#tfds.list_builders()

## Data Preprocessing

In [3]:
# Data loading
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [4]:
# Data extracting and building references
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# Definition of the number of validation samples in the variable
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

# Cast this number to integer
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# Definition of the number of test samples in the variable (instead of using the mnist_info)
num_test_samples = mnist_info.splits['test'].num_examples

# Cast this number to integer
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Data scaling for MNIST image and its label
# Function creation
def scale (image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

# Data transformation
scaled_train_and_validation_data = mnist_train.map(scale)

# Test data scaling
test_data = mnist_test.map(scale)

# Data shuffling
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# Extracting the validation data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# Extracting the train data
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Data batching
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

# Loading of data into variables through data iterations
validation_inputs, validation_targets = next(iter(validation_data))

## Building the Model

### Model Outlining

In [5]:
# The model outlining
input_size = 784
output_size = 10

# Optional hidden layers size
hidden_layer_size = 200

# Definition of the model look
model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2st hidden layer
    
                            tf.keras.layers.Dense(output_size, activation='softmax'), # Output layer    
])

### Selection of the optimizer and loss function

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Model Training

In [7]:
NUM_EPOCHS = 7

# Model training using the training data, specified number of epochs, and the validation data
model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose = 2)

Epoch 1/7
540/540 - 9s - loss: 0.2753 - accuracy: 0.9212 - val_loss: 0.1426 - val_accuracy: 0.9580
Epoch 2/7
540/540 - 7s - loss: 0.1048 - accuracy: 0.9683 - val_loss: 0.0928 - val_accuracy: 0.9728
Epoch 3/7
540/540 - 8s - loss: 0.0724 - accuracy: 0.9771 - val_loss: 0.0765 - val_accuracy: 0.9775
Epoch 4/7
540/540 - 8s - loss: 0.0526 - accuracy: 0.9834 - val_loss: 0.0605 - val_accuracy: 0.9820
Epoch 5/7
540/540 - 8s - loss: 0.0404 - accuracy: 0.9872 - val_loss: 0.0513 - val_accuracy: 0.9830
Epoch 6/7
540/540 - 8s - loss: 0.0323 - accuracy: 0.9896 - val_loss: 0.0399 - val_accuracy: 0.9873
Epoch 7/7
540/540 - 8s - loss: 0.0253 - accuracy: 0.9921 - val_loss: 0.0321 - val_accuracy: 0.9902


### Model Testing

In [8]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0667 - accuracy: 0.9809


In [9]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.0))

Test loss: 0.07. Test accuracy: 98.09%
